In [1]:
# Import library
import PyPDF2
import io
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import csv
pdfmetrics.registerFont(TTFont('Crimson-Regular', 'CrimsonText-Regular.ttf'))

# Copyright@ Xu (Aaron) Weng

In [21]:
# Function to fill the PDF template with data
def fill_pdf_template(pdf_template_path, output_pdfs, row_index, row):
    output_pdf_path = f"{output_pdfs}/Admin_{row_index}.pdf"
    
    pdf_reader = PyPDF2.PdfReader(open(pdf_template_path, "rb"))
    pdf_writer = PyPDF2.PdfWriter()

    # Read template
    content_page = pdf_reader.pages[0]
    mediabox = content_page.mediabox
    
    # New a pdf with words
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=(mediabox.width, mediabox.height))
    name = row[0]
    title = row[1]

    # locations of words need to be customized
    x_coordinate1 = round(mediabox.width/2 - len(name)/2*30) #1100
    y_coordinate1 = 1650

    x_coordinate2 = round(mediabox.width/2 - len(title)/2*30)-50 #800
    y_coordinate2 = 1160


    can.setFont("Crimson-Regular", 80)  # Customize the font and size
    can.drawString(x_coordinate1, y_coordinate1, name)
    can.drawString(x_coordinate2, y_coordinate2, title)
    can.save()    
    #move to the beginning of the StringIO buffer
    packet.seek(0)
    new_pdf = PyPDF2.PdfReader(packet)

    # Merge two pdf  
    content_page.merge_page(new_pdf.pages[0])
    content_page.mediabox = mediabox
    pdf_writer.add_page(content_page)
    # pdf_writer.add_page(new_pdf.pages[0])

    # finally, write "output" to a real file
    output_stream = open(output_pdf_path, "wb")
    pdf_writer.write(output_stream)
    output_stream.close()

In [ ]:
# Read data from the CSV file and generate filled PDFs
csv_filename = "Admin.csv"  # Replace with your CSV file path
template_pdf_path = "template.pdf"  # Replace with your PDF template file path
output_folder = "Admin"  # Replace with the desired output folder

with open(csv_filename, 'r') as csv_filename:
        csvreader = csv.reader(csv_filename)
        for row_index, row in enumerate(csvreader, start=1):
            fill_pdf_template(template_pdf_path, output_folder, row_index, row)

print("Filled PDFs created successfully.")